In [ ]:
! pip install environs facebook_business cyksuid

In [11]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
import os
from environs import Env
import requests
env = Env()
env.read_env('.other-env')

cnf = {
    'APP_ID': env('FACEBOOK_APP_ID'),
    'APP_SECRET': env('FACEBOOK_APP_SECRET'),
    'PAGE_ID': env('FACEBOOK_PAGE_ID'),
    'PAGE_TOKEN': env('FACEBOOK_PAGE_TOKEN'),
    'USER_TOKEN': env('FACEBOOK_USER_TOKEN'),
    'AD_ACCOUNT': f'act_{env("FACEBOOK_AD_ACCOUNT")}'
}


In [ ]:
FacebookAdsApi.init(cnf['APP_ID'], cnf['APP_SECRET'], cnf['USER_TOKEN'])

account = AdAccount(cnf['AD_ACCOUNT'])

In [62]:
# from responses import query

# q = """
# SELECT distinct(userid) FROM responses
# """

# res = query('chatroach', 'chatreader', q)
# users = [i[0] for i in res]

In [57]:
from facebook_business.adobjects.customaudience import CustomAudience
import re
import requests
import json

def add_users(aud_id, pageid, users):
    url = f'https://graph.facebook.com/v7.0/{aud_id}/users?access_token={cnf["USER_TOKEN"]}'

    json = {
        'payload': {
            'schema': ['PAGEUID'],
            'is_raw': True,
            'page_ids': [pageid],
            'data': [[u] for u in users]
        }
    }

    res = requests.post(url, json=json)
    return res.json()


def create_custom_audience(account, pageid, name, desc, users):
    params = {
        'name': name,
        'subtype': 'CUSTOM',
        'description': desc,
        'customer_file_source': 'USER_PROVIDED_ONLY',
    }

    aud = account.create_custom_audience(fields=[], params=params)
    res = add_users(aud.get_id(), pageid, users)
    return aud, res



def create_lookalike(account, name, country, custom_audience):

    spec = {
        'starting_ratio': 0.01,
        'ratio': 0.10,
        'country': country
    }


    params = {
        CustomAudience.Field.name: name,
        CustomAudience.Field.subtype: CustomAudience.Subtype.lookalike,
        CustomAudience.Field.origin_audience_id: custom_audience.get_id(),
        CustomAudience.Field.lookalike_spec:json.dumps(spec),
    }

    return account.create_custom_audience(params=params)


def get_all_audiences(account, pat=r'vlab-'):
    fields = [CustomAudience.Field.name,
              CustomAudience.Field.description, 
              CustomAudience.Field.subtype,
              CustomAudience.Field.time_created]


    audiences = account.get_custom_audiences(fields=fields)

    audiences = [aud for aud in audiences 
                 if re.match(pat, aud['name'])]

    audiences = [aud for aud in audiences
                 if aud['subtype'] == CustomAudience.Subtype.custom]

    audiences = sorted(audiences, key=lambda x: -x['time_created'])
    return audiences

In [448]:

latest_aud = get_all_audiences(account, '')[0]
lookalike = create_lookalike(account, 'api-lookalike-2', 'IN', latest_aud)

/home/jupyter/.local/lib/python3.8/site-packages/facebook_business/utils/api_utils.py:30: UserWarning: value of lookalike_spec might not be compatible.  Expect string;  got <class 'dict'>
  warnings.warn(message)


In [204]:
from facebook_business.adobjects.adimage import AdImage
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.adcreative import AdCreative

import json

def get_images(account):
    return account.get_ad_images(fields=[AdImage.Field.hash, 
                                         AdImage.Field.name,
                                         AdImage.Field.created_time,
                                         AdImage.Field.filename])


def get_image_hash(account, name):
    images = get_images(account)
    img = next((img for img in images if img['name'] == name), None)
    if not img:
        raise Exception(f'Could not find image with name: {name}')
    
    return img['hash']

def make_welcome_message(text, button_text, form):
    payload = json.dumps({'referral': { 'ref': f'form.{form}'}})
    
    message = {
        "message": {
            "attachment": {
                "type": "template",
                "payload": {
                    "template_type": "button",
                    "text": text,
                    "buttons": [
                        {
                            "type": "postback",
                            "title": "Start!",
                            "payload": payload
                        }
                    ]        
                }
            }
        }
    }

    return json.dumps(message)
    

def create_creative(account, pageid, config):
    image_hash = get_image_hash(account, config['image'])    
    link_text = config['link_text']
    form = config['form']
    msg = make_welcome_message(config['welcome_message'], config['button_text'], form)

    oss = {
        "link_data": {
            "call_to_action": {
                "type": "MESSAGE_PAGE",
                "value": {
                    "app_destination": "MESSENGER"
                }
            },
            "image_hash": image_hash,
            "message": config['body'],
            "name": config['link_text'],
            "page_welcome_message": msg
        },
        "page_id": pageid,

    }

    params = {
        "name": config['name'],
        "url_tags": f"ref=form.{form}",
        "actor_id": pageid,
        "object_story_spec": oss
    }

    return account.create_ad_creative(fields=['name', 'object_story_spec'], params=params)



In [ ]:
# create_custom_audience('uid-test-audience', 'testing for uid sdk usage', users[:10000])

In [265]:
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.targetinggeolocationcustomlocation import TargetingGeoLocationCustomLocation
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from datetime import datetime, timedelta
from cyksuid import ksuid

def create_location(lat, lng, rad):
    return {
        TargetingGeoLocationCustomLocation.Field.latitude: lat,
        TargetingGeoLocationCustomLocation.Field.longitude: lng,
        TargetingGeoLocationCustomLocation.Field.radius: rad,
        TargetingGeoLocationCustomLocation.Field.distance_unit: 'kilometer',
    }

def create_adset(account, name, custom_locs, campaign_id, budget, status, audience=None):
    targeting = {'geo_locations': { TargetingGeoLocation.Field.custom_locations: custom_locs}}
    
    if audience:
        targeting['custom_audiences'] = [{'id': audience['id']}]

    params = {
        AdSet.Field.name: name,
        AdSet.Field.lifetime_budget: budget,
        AdSet.Field.start_time: datetime.utcnow() + timedelta(minutes=5),
        AdSet.Field.end_time: datetime.utcnow() + timedelta(days=1),
        AdSet.Field.campaign_id: campaign_id,
        AdSet.Field.optimization_goal: AdSet.OptimizationGoal.replies,
        AdSet.Field.billing_event: AdSet.BillingEvent.impressions,
        AdSet.Field.bid_strategy: AdSet.BidStrategy.lowest_cost_without_cap,
        AdSet.Field.targeting: targeting,
        AdSet.Field.status: status
    }

    adset = account.create_ad_set(params=params)

    return adset

def create_ad(account, adset, creative, name, status):
    return account.create_ad(params={
        'name': name,
        'status': status,
        'adset_id': adset['id'],
        'creative': { 'creative_id': creative['id']}
    })

def create_ads(account, adset, creatives, status):
    ads = [create_ad(account, adset, c, c['name'], status)
           for c in creatives]

    return ads

def get_creatives(account, ids):
    
    creatives = account.get_ad_creatives(fields=['name', 'object_story_spec'])
    
    return [c for c in creatives if c['id'] in ids]


def launch_adset(account, campaign, creatives, locs, budget, status=AdSet.Status.paused):
    uid = ksuid.ksuid().encoded.decode('utf-8')
    name = f'vlab-{uid}'
    
    custom_locs = [create_location(lat, lng, rad) 
            for lat, lng, rad in locs]


    adset = create_adset(account, name, custom_locs, 
                         campaign['id'], budget, status)

    creatives = get_creatives(account, creatives)

    if len(creatives) == 0:
        raise Exception('No creatives!!')

    ads = create_ads(account, adset, creatives, status)

    return adset, ads


def get_campaign(account, name):
    campaigns = account.get_campaigns(fields=['name'])
    c = next((c for c in campaigns if c['name'] == name), None)
    if not c:
        raise Exception(f'Could not find campaign: {name}')
    return c



In [209]:
campaign = get_campaign(account, 'impact-evaluation-1')

campaign

<Campaign> {
    "id": "23844995397410518",
    "name": "impact-evaluation-1"
}

In [211]:
import pandas as pd
cities = pd.read_csv('output/cities.csv')
cities = cities[cities.rad >= 1.0]

locs = [(r.lat, r.lng, r.rad) for _, r 
        in cities[:200].iterrows()]

cities.shape

(82, 5)

In [266]:
status = AdSet.Status.active
res = launch_adset(account, campaign, [creative['id']], locs, 700000, status=status)

In [256]:
def load_creatives():
    with open('creatives.json') as f:
        s = f.read()

    d = json.loads(s)
    d = d['creatives']

    return d


c = load_creatives()[0]
creative = create_creative(account, cnf['PAGE_ID'], c)